In [1]:
from zipfile import ZipFile
import os
import pandas as pd
from datetime import datetime, date
import glob
import win32com.client as win32
import os
import shutil
from openpyxl import load_workbook
from pywintypes import com_error
import numpy as np
import time
import base64
import pyodbc

In [2]:
# !pip install xlsxwriter

In [3]:
#adjust the number of days to offset the report date and the previous report date
k = 1
l = 2

In [4]:
################# Step 1. get the correct date ################################
# Create an offset of 1 Business days
offset_today = pd.tseries.offsets.BusinessDay(n=k)
offset_previous_day = pd.tseries.offsets.BusinessDay(n=l)

# initializing dates
today_date = date.today() - offset_today
prior_date = date.today() - offset_previous_day

# printing original date
print("Date's report date is: " + str(today_date))
print("Previous report date is: " + str(prior_date))

# Creating Timestamp
ts = pd.Timestamp(str(today_date))

Date's report date is: 2024-12-04 00:00:00
Previous report date is: 2024-12-03 00:00:00


In [5]:
reportdate = today_date.strftime('%Y%m%d')
prior_reportdate = prior_date.strftime('%Y%m%d')

In [6]:
savePath = "C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Active FX Performance\\"
os.chdir(savePath)

In [7]:
file_list = ['Aggregated Currency Exposure_IMCO GEP_'+reportdate+'_CM&TP.xlsx',
             #'Aggregated Currency Exposure_IMCO GEP_'+reportdate+'_Record.xlsx',
             'Aggregated Currency Exposure_DoD_'+reportdate+'.xlsx']

In [8]:
prior_file_list = ['Aggregated Currency Exposure_IMCO GEP_'+prior_reportdate+'_CM&TP.xlsx',
             #'Aggregated Currency Exposure_IMCO GEP_'+prior_reportdate+'_Record.xlsx',
             'Aggregated Currency Exposure_DoD_'+prior_reportdate+'.xlsx']

In [9]:
for i in range(len(file_list)):

    # construct full file path
    destination = savePath + file_list[i]
    source = savePath + prior_file_list[i]

    shutil.copy(source, destination)
    print('copied', file_list[i])

copied Aggregated Currency Exposure_IMCO GEP_20241204_CM&TP.xlsx
copied Aggregated Currency Exposure_DoD_20241204.xlsx


In [10]:
reportdate2 = reportdate[0:4]+"-"+reportdate[4:6]+"-"+reportdate[6:8]

def closeRate(date):
    conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=172.16.50.4;'
                      'Database=PA;'
                      'Trusted_Connection=yes;')
    cursor=conn.cursor()

    rates = pd.read_sql("select ctr_ccy, close_rate, effective_date from Master_DM..fx_rate where ctr_ccy in ('AUD','CHF','DKK','EUR','GBP','HKD','ILS','JPY','NOK','NZD','SEK','SGD','USD') and effective_date = '{}'".format(date), conn)   
    cursor.commit()
    cursor.close()
    conn.close()
    
    return rates

rates = closeRate(reportdate2)

C:\Users\JiRoxanna\AppData\Local\Temp\ipykernel_6592\1958831923.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rates = pd.read_sql("select ctr_ccy, close_rate, effective_date from Master_DM..fx_rate where ctr_ccy in ('AUD','CHF','DKK','EUR','GBP','HKD','ILS','JPY','NOK','NZD','SEK','SGD','USD') and effective_date = '{}'".format(date), conn)


In [11]:
rates

,ctr_ccy,close_rate,effective_date
0,AUD,1.105090,2024-12-04
1,CHF,0.628239,2024-12-04
2,DKK,5.035406,2024-12-04
3,EUR,0.675207,2024-12-04
4,GBP,0.559436,2024-12-04
5,HKD,5.534677,2024-12-04
6,ILS,2.570367,2024-12-04
7,JPY,106.949629,2024-12-04
8,NOK,7.849988,2024-12-04
9,NZD,1.213138,2024-12-04


In [12]:
x = rates.to_excel('C:/O365Sync/IMCO/RISK Analytics - Documents/FX Analytics/rates.xlsx', sheet_name='sheet1', index=False)

In [13]:
import openpyxl as xl;

xl = win32.Dispatch("Excel.Application")  # instantiate excel app
xl.Application.visible = True
xl.DisplayAlerts = True

wb1 = xl.Workbooks.Open('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\rates.xlsx')
wb2 = xl.Workbooks.Open('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Record Outstanding Currency Exposures.xlsx')
wb3 = xl.Workbooks.Open('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Active FX Performance\\BBGAIM_FX_EOD_OpenPosition_'+ reportdate + '.csv')
wb4 = xl.Workbooks.Open('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\IMCO Global EQ FX Exposure_Master_2021_v1.xlsm')
wb5 = xl.Workbooks.Open('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Active FX Performance\\IMCO Global EQ FX Exposure_'+ prior_reportdate + '.xlsx')


ws1 = wb1.Worksheets('sheet1')
ws2 = wb2.Worksheets('Sheet2')
ws3 = wb3.Worksheets('BBGAIM_FX_EOD_OpenPosition_2024')
ws4 = wb2.Worksheets('Outstanding Trades')
ws5 = wb4.Worksheets('Exposure')
ws6 = wb5.Worksheets('Exposure')

ws1.Activate()
ws1.Range('B2:B14').Copy()
ws2.Activate()
ws2.Range('E2:E14').Select()
xl.Selection.PasteSpecial(Paste=12)

ws3.Activate()
ws3.Range('A:N').Copy()
ws4.Activate()
ws4.Range('A:N').Select()
xl.Selection.PasteSpecial(Paste=12)

ws2.Activate()
ws2.Range('F2:F14').Copy()
ws5.Activate()
ws5.Range('B40:B52').Select()
xl.Selection.PasteSpecial(Paste=12)

ws6.Activate()
ws6.Range('E5:E19').Copy()
ws5.Activate()
ws5.Range('J5:J19').Select()
xl.Selection.PasteSpecial(Paste=12)

ws5.Range('B1').Value = reportdate2


In [14]:
xl.Application.Run('\'' + "IMCO Global EQ FX Exposure_Master_2021_v1.xlsm"+ '\'' +"!ExposureRecon.GetDatafromSQL")


# Stop to check if any redemption

In [15]:
# saving the destination excel file
wb1.Save()
wb1.Close()
wb2.Save()
wb2.Close()
wb3.Close()
wb5.Close()
wb4.Save()

In [16]:
sharepoint_path = 'C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Active FX Performance\\'
sharepoint_filename = 'IMCO Global EQ FX Exposure_'+ reportdate + '.xlsx'
# save as xlsx
wb4.SaveAs(Filename=os.path.join(sharepoint_path, sharepoint_filename), FileFormat=51) # xlOpenXMLWorkbook	51	Open XML Workbook	*.xlsx

In [17]:
# check if there is any subscription or redemption; if so, adjust accordingly. If not, move on to the next step

In [18]:
wb6 = xl.Workbooks.Open('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Active FX Performance\\Aggregated Currency Exposure_IMCO GEP_'+reportdate+'_CM&TP.xlsx')
#wb7 = xl.Workbooks.Open('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Active FX Performance\\Aggregated Currency Exposure_IMCO GEP_'+reportdate+'_Record.xlsx')
wb8 = xl.Workbooks.Open('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Active FX Performance\\Aggregated Currency Exposure_DoD_'+reportdate+'.xlsx')

In [19]:
ws7 = wb6.Worksheets('Sheet1')
#ws8 = wb7.Worksheets('Sheet1')
ws9 = wb8.Worksheets('Current')
ws10 = wb8.Worksheets('Previous')

In [20]:
ws9.Activate()
ws9.Range('A2:D17').Copy()
ws10.Activate()
ws10.Range('A2:D17').Select()
xl.Selection.PasteSpecial(Paste=12)

#ws5.Activate()
#ws5.Range('B5:E20').Copy()
#ws8.Activate()
#ws8.Range('A2:D17').Select()
#xl.Selection.PasteSpecial(Paste=12)

ws5.Activate()
ws5.Range('B5:E20').Copy()
ws9.Activate()
ws9.Range('A2:D17').Select()
xl.Selection.PasteSpecial(Paste=12)

ws5.Activate()
ws5.Range('B5:G20').Copy()
ws7.Activate()
ws7.Range('A2:F17').Select()
xl.Selection.PasteSpecial(Paste=12)

True

In [21]:
wb6.Save()
wb6.Close()
#wb7.Save()
#wb7.Close()

In [22]:
ws5.Activate()
ws5.Range('B1').Select()
wb4.Save()

In [23]:
ws11 = wb8.Worksheets('Summary')
ws12 = wb4.Worksheets('Exposure')
ws13 = wb8.Worksheets('Diff Summary')
ws14 = wb4.Worksheets('Data')

ws12.Activate()
ws12.Range('A5:J19').Copy()
ws11.Activate()
ws11.Range('A3:J17').Select()
xl.Selection.PasteSpecial(Paste=12)

ws13.Activate()
ws13.Range('A2:D17').Copy()
ws11.Activate()
ws11.Range('A21:D36').Select()
xl.Selection.PasteSpecial(Paste=12)

ws12.Activate()
ws12.Range('A24:D36').Copy()
ws11.Activate()
ws11.Range('A40:D52').Select()
xl.Selection.PasteSpecial(Paste=12)

ws9.Activate()
ws9.Range('H2:J4').Copy()
ws11.Activate()
ws11.Range('B56:D58').Select()
xl.Selection.PasteSpecial(Paste=12)

ws14.Activate()
ws14.Range('T3:W16').Copy()
ws11.Activate()
ws11.Range('A63:D76').Select()
xl.Selection.PasteSpecial(Paste=12)

wb8.Save()
wb8.Close()

In [24]:
wb_dod = xl.Workbooks.Open('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Active FX Performance\\Aggregated Currency Exposure_DoD_'+reportdate+'.xlsx')

In [ ]:
# Check if any diff of FX larger than 10MM - if so, continue

# If EM, manual check

In [224]:
# =VLOOKUP(B2,Exposure!$B$5:$B$18,1,0)

In [225]:
df = pd.read_excel('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Active FX Performance\\Aggregated Currency Exposure_DoD_'+reportdate+'.xlsx', sheet_name='Diff Summary',header=0)

In [226]:
df = df[:15]

In [227]:
list(df[(abs(df['ACE (CAD $)'])>=10000000) & (abs(df['AFX (CAD $)'])>=10000000)]['Currency'])

['USD']

In [228]:
reportdate2 = reportdate[0:4]+"-"+reportdate[4:6]+"-"+reportdate[6:8]
prior_reportdate2 = prior_reportdate[0:4]+"-"+prior_reportdate[4:6]+"-"+prior_reportdate[6:8]

def diff(date):
    conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=172.16.50.4;'
                      'Database=PA;'
                      'Trusted_Connection=yes;')
    cursor=conn.cursor()

    di = pd.read_sql("SELECT * FROM \
    (SELECT \
    case  \
    when s.user_group_sector14 is null then s.Currency_Code \
    when s.User_Group_Sector14 <> '003' then s.Currency_Code \
    else cc.Currency_Code \
    end as Currency_Code \
    ,PortCode \
    ,PortName \
    , SUM(CASE WHEN r.GrossEndMktVal IS NULL THEN 0 ELSE r.GrossEndMktVal END) AS MV_END, \
    r.end_effective_date \
    FROM Master_DM..Perf_Return r, \
    Master_DM..Security_Security_Master s, \
    PA..COUNTRY_CODE cc, \
    PA..PORTFOLIO p \
    WHERE s.Security_Alias = r.security_id \
    AND r.entity_id = p.PortCode \
    AND p.PortGroup = 'GEP_CURR' \
    AND p.Date_ = (SELECT MAX(p1.Date_) FROM PA..PORTFOLIO p1 WHERE p1.PortGroup = p.PortGroup AND p1.PortCode = p.PortCode AND p1.Date_ <= r.end_effective_date) \
    AND s.Nra_Tax_Country = cc.COUNTRY_CODE \
    AND r.end_effective_date = '{}' \
    AND (s.User_Group_Sector14 IN ('003', '006', '600','004','INFR') or s.User_Group_Sector14 is NULL or (s.User_Group_Sector14 in ('007') and Issue_Name = 'Project Cloud')) \
    AND r.dictionary_id = 0 \
    AND r.perf_freq_code = 'D' \
    and r.data_source = 'PACE' \
    and s.Primary_Asset_Id <> 'IMCGPSUBCL01' \
    GROUP BY \
    r.begin_effective_date \
    , r.end_effective_date \
    , r.src_intfc_inst \
    , r.perf_freq_code \
    , r.dictionary_id \
    , PortCode \
    , PortName \
    , case  \
    when s.user_group_sector14 is null then s.Currency_Code \
    when s.User_Group_Sector14 <> '003' then s.Currency_Code \
    else cc.Currency_Code \
    end) A \
    WHERE A.Currency_Code in ('USD') order by currency_code asc, portcode asc".format(date), conn)   
    cursor.commit()
    cursor.close()
    conn.close()
    
    return di

di0 = diff(prior_reportdate2)
di1 = diff(reportdate2)

C:\Users\JiRoxanna\AppData\Local\Temp\ipykernel_4856\3645314246.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  di = pd.read_sql("SELECT * FROM \


In [229]:
di = pd.merge(di0,di1, how='right', on=['Currency_Code', 'PortCode', 'PortName'])
di = di.drop(columns = ['end_effective_date_x', 'end_effective_date_y'])
di['Diff'] = di['MV_END_y'] - di['MV_END_x']
di.columns = ['Currency_Code', 'PortCode', 'PortName', prior_reportdate2, reportdate2, 'Diff'] 
di = di[(abs(di['Diff'])>999999)]

In [230]:
di

,Currency_Code,PortCode,PortName,2024-12-02,2024-12-03,Diff
3,USD,IMP022D,IMCO GE Index Completion,1.851786e+09,1.863153e+09,1.136757e+07
4,USD,IMP022E,IMCO Baillie G Global Equity,4.507220e+08,4.539729e+08,3.250870e+06
5,USD,IMP022F,IMCO GQG Global Equity,6.405594e+08,6.433100e+08,2.750560e+06
6,USD,IMP022G,IMCO Guardian C Global Equity,4.405124e+08,4.373324e+08,-3.180019e+06
15,USD,IMP022R,IMCO GPE LA Capital,8.712549e+08,8.735225e+08,2.267541e+06
19,USD,IMP022V,IMCO ARGA Global Conc EQ,3.350090e+08,3.323720e+08,-2.636936e+06
22,USD,IMP02AA,IMCO GPE Liquidity,2.490581e+09,2.504004e+09,1.342266e+07


In [231]:
#writer = pd.ExcelWriter('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Active FX Performance\\DoD_'+reportdate+'.xlsx', 
                        #engine='xlsxwriter')
#di.to_excel(writer, sheet_name="DoD Change", index=False)

In [232]:
#DoD = writer.book
#DoD_sheet = writer.sheets['DoD Change']
#format1 = DoD.add_format({'num_format': '###,###,###,###'})

In [233]:
#DoD_sheet.set_column("D:F", 14, format1)
#DoD_sheet.set_column("A:A", 15)
#DoD_sheet.set_column("B:B", 9)
#DoD_sheet.set_column("C:C", 30)

In [234]:
#writer.close()

In [235]:
di.to_excel('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Active FX Performance\\DoD1_'+reportdate+'.xlsx', sheet_name="DoD Change", index=False)

In [236]:
xl = win32.Dispatch("Excel.Application")  # instantiate excel app
xl.Application.visible = True
xl.DisplayAlerts = True

DoD = xl.Workbooks.Open('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Active FX Performance\\DoD1_'+reportdate+'.xlsx')
DoD_sheet = DoD.Worksheets['DoD Change']

# manually change the format

In [237]:
#wb_dod = xl.Workbooks.Open('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Active FX Performance\\Aggregated Currency Exposure_DoD_'+reportdate+'.xlsx')

In [238]:
#diff_dod = xl.Workbooks.Open('C:\\O365Sync\\IMCO\\RISK Analytics - Documents\\FX Analytics\\Active FX Performance\\DoD_'+reportdate+'.xlsx')

In [ ]:
#The DoD change of AFX in USD was mainly from the accounts below.